In [1]:
# SQLAlchemy, Sqlite, and Dates

## Setup

In [2]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [3]:
engine = create_engine("sqlite:///../Resources/dow.sqlite", echo=False)

## Explore Database

In [4]:
inspector = inspect(engine)
inspector.get_table_names()

['dow']

In [5]:
# Get a list of column names and types
columns = inspector.get_columns('dow')
for c in columns:
    print(c['name'], c["type"])
# columns

id INTEGER
quarter INTEGER
stock TEXT
date TEXT
open_price FLOAT
high_price FLOAT
low_price FLOAT
close_price FLOAT
volume INTEGER
percent_change FLOAT


In [6]:
engine.execute('SELECT * FROM dow LIMIT 5').fetchall()

[(1, 1, 'AA', '2011-01-07', 15.82, 16.72, 15.78, 16.42, 239655616, None),
 (2, 1, 'AA', '2011-01-14', 16.71, 16.71, 15.64, 15.97, 242963398, None),
 (3, 1, 'AA', '2011-01-21', 16.19, 16.38, 15.6, 15.79, 138428495, None),
 (4, 1, 'AA', '2011-01-28', 15.87, 16.63, 15.82, 16.13, 151379173, None),
 (5, 1, 'AA', '2011-02-04', 16.18, 17.39, 16.18, 17.14, 154387761, None)]

## Reflect and query dates

In [7]:
# Reflect Database into ORM class
Base = automap_base()
Base.prepare(engine, reflect=True)
Dow = Base.classes.dow

In [8]:
session = Session(engine)

How many dates do we have?

In [9]:
# Total dates
session.query(func.count(Dow.date)).all()

[(750)]

What is the earliest date?

In [10]:
# Earliest Date
session.query(Dow.date).order_by(Dow.date).first()

('2011-01-07')

What is the latest date?

In [11]:
# Latest Date
session.query(Dow.date).order_by(Dow.date.desc()).first()

('2011-06-24')

Find all of the dates great than `2011-03-01`

In [12]:
session.query(Dow.date).\
    filter(Dow.date > '2011-03-01').\
    order_by(Dow.date).all()

[('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'

### Important Note! Sqlite does not support a date column type, but SQLAlchemy will allow you to work with dates in the iso format. [sqlite dates](http://docs.sqlalchemy.org/en/latest/dialects/sqlite.html)

# Quick Review of DateTime

In [13]:
import datetime as dt

In [14]:
# Print today's date 
print(dt.date.today())
# Print a specific date 
print(dt.date(2017,10 ,8))

2021-01-14
2017-10-08


In [15]:
# Print a datetime object 
print(dt.datetime.utcnow())
print(dt.datetime(2017,10 ,8))

2021-01-14 18:22:33.157431
2017-10-08 00:00:00


Calculate a time delta

In [16]:
# The date 9 weeks ago from today. 
week_ago = dt.date.today() - dt.timedelta(days=63)
week_ago

datetime.date(2020, 11, 12)

In [17]:
# The number of days between two datetime objects – When the next solar eclipse will be and today.
a = dt.datetime(2023, 10, 14) 
b = dt.datetime.utcnow()
  
# returns a timedelta object 
c = a-b  
print(c)

1002 days, 5:37:26.832448


Query for the Dow closing price 1 week before `2011-04-08` using the datetime library

In [18]:
# Query for the Dow closing price for `CSCO` 1 week before `2011-04-08` using the datetime library
query_date = dt.date(2011, 4, 8) - dt.timedelta(days=7)
print("Query Date: ", query_date)

Query Date:  2011-04-01


In [19]:
session.query(Dow.date, Dow.close_price).\
    filter(Dow.stock == 'CSCO').\
    filter(Dow.date == query_date).all()

[('2011-04-01', 17.04)]

In [20]:
# Parse out just the day from the datetime object
dt.date.today().strftime("%d")

'14'

Putting it all together

In [21]:
# Query for all stocks, their opening and closing price that match
# following date string in the format `%d`
date_str = "14"
session.query(Dow.date, Dow.stock, Dow.open_price, Dow.close_price).\
    filter(func.strftime("%d", Dow.date) == date_str).all()

[('2011-01-14', 'AA', 16.71, 15.97),
 ('2011-01-14', 'AXP', 44.2, 46.25),
 ('2011-01-14', 'BA', 69.42, 70.07),
 ('2011-01-14', 'BAC', 14.17, 15.25),
 ('2011-01-14', 'CAT', 93.21, 94.01),
 ('2011-01-14', 'CSCO', 20.94, 21.21),
 ('2011-01-14', 'CVX', 90.95, 92.83),
 ('2011-01-14', 'DD', 48.3, 49.8),
 ('2011-01-14', 'DIS', 39.01, 39.29),
 ('2011-01-14', 'GE', 18.61, 18.82),
 ('2011-01-14', 'HD', 34.16, 35.89),
 ('2011-01-14', 'HPQ', 44.86, 46.25),
 ('2011-01-14', 'IBM', 147.0, 150.0),
 ('2011-01-14', 'INTC', 20.71, 21.08),
 ('2011-01-14', 'JNJ', 62.29, 62.55),
 ('2011-01-14', 'JPM', 43.27, 44.91),
 ('2011-01-14', 'KRFT', 30.91, 31.34),
 ('2011-01-14', 'KO', 62.7, 63.13),
 ('2011-01-14', 'MCD', 74.25, 74.06),
 ('2011-01-14', 'MMM', 85.7, 88.1),
 ('2011-01-14', 'MRK', 37.26, 34.23),
 ('2011-01-14', 'MSFT', 28.2, 28.3),
 ('2011-01-14', 'PFE', 18.22, 18.34),
 ('2011-01-14', 'PG', 64.4, 65.53),
 ('2011-01-14', 'T', 28.54, 28.43),
 ('2011-01-14', 'TRV', 53.12, 54.63),
 ('2011-01-14', 'UTX', 78.

## Your Turn!

DataSet Citation: 

Brown, M. S., Pelosi, M. & Dirska, H. (2013). Dynamic-radius Species-conserving Genetic Algorithm for 
the Financial Forecasting of Dow Jones Index Stocks. Machine Learning and Data Mining in Pattern 
Recognition, 7988, 27-41.
